In [78]:
import pandas as pd
import tensorflow as tf
from datasets import load_dataset

In [79]:
# load yelp review dataset
dataset = load_dataset('yelp_polarity', split='train')
dataset

Dataset yelp_polarity downloaded and prepared to /Users/alexanderkell/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 560000
})

In [82]:
# print first 5 rows
type(dataset)


datasets.arrow_dataset.Dataset

In [83]:
from transformers import AutoTokenizer, TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
bert = TFAutoModel.from_pretrained("distilbert-base-cased")

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [95]:
tokenize_dataset = dataset.map(lambda x: tokenizer.encode_plus(x['text'], max_length=SEQ_LEN, truncation=True,
                        pad_to_max_length=True, add_special_tokens=True,
                        return_attention_mask=True, return_token_type_ids=False,
                        return_tensors='tf'))
                        


TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

In [105]:
len(tokenize_dataset['input_ids'])

560000

In [90]:
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

In [91]:
embeddings = bert(input_ids, attention_mask=mask)[0]  # we only keep tensor 0 (last_hidden_state)

X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(X)  # 2 labels due to three sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

In [92]:
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [108]:
model.fit({"input_ids": tokenize_dataset['input_ids'], "attention_mask": tokenize_dataset['attention_mask']},
          labels=tokenize_dataset['label'])

TypeError: fit() got an unexpected keyword argument 'labels'